In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
def load_weather_data(file_path, sheet_name):
    """
    Loads weather data from an Excel sheet, keeping only 'time' and 'tavg' columns.

    Parameters:
    file_path (str): The path to the Excel file.
    sheet_name (str): The sheet name in the Excel file.

    Returns:
    DataFrame: A DataFrame with 'time' as the index and 'tavg' as the single column.
    """
    # Load data from specified sheet
    df = pd.read_excel(file_path, sheet_name=sheet)

    # Keep only the 'time' and 'tavg' columns and set 'time' as index
    df = df[['time', 'tavg']]
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)

    return df

# Đường dẫn tới file Excel
file_path = "data_thoitiet.xlsx"

# Đọc toàn bộ các sheet
excel_data = pd.ExcelFile(file_path)

# Lấy danh sách các tên sheet
sheet_names = excel_data.sheet_names
# Duyệt qua từng sheet và tải dữ liệu
for sheet in sheet_names:
    df = load_weather_data(file_path, sheet_name=sheet)
    train_size = int(len(df) * 0.8)
    train_data = df[:train_size]
    test_data = df[train_size:]

    # Bước 3: Chuẩn hóa dữ liệu
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train_data = scaler.fit_transform(train_data)
    scaled_test_data = scaler.transform(test_data)

    # Bước 4: Tạo tập dữ liệu để huấn luyện mô hình
    def create_dataset(data, time_step=1):
        x, y = [], []
        for i in range(len(data) - time_step - 1):
            a = data[i:(i + time_step), 0]
            x.append(a)
            y.append(data[i + time_step, 0])
        return np.array(x), np.array(y)

    # Thay đổi kích thước dữ liệu
    time_step = 30
    x_train, y_train = create_dataset(scaled_train_data, time_step)
    x_test, y_test = create_dataset(scaled_test_data, time_step)

    # Reshape dữ liệu cho LSTM [samples, time steps, features]
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

    # Bước 5: Xây dựng mô hình LSTM
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=64))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Biên dịch mô hình
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Bước 6: Huấn luyện mô hình
    save_model_path = f'/content/drive/MyDrive/Cap2/{sheet}_best_weather_model.keras'
    checkpoint = ModelCheckpoint(save_model_path, monitor='loss', save_best_only=True, mode='auto')
    model.fit(x_train, y_train, epochs=100, batch_size=32, callbacks=checkpoint)
